The first example we'll look at is that of thermal emission from a galaxy cluster. In this case, the gas in the core of the cluster is "sloshing" in the center, producing spiral-shaped cold fronts. The dataset we want to use for this example is available for download from the [yt Project](http://yt-project.org) at [this link](http://yt-project.org/data/GasSloshing.tar.gz). 

First, import our necessary modules:

In [ ]:
%matplotlib inline
import yt
import pyxsim
import yt.units as u
import matplotlib.pyplot as plt

Next, we `load` the dataset with yt:

In [ ]:
ds = yt.load("GasSloshing/sloshing_nomag2_hdf5_plt_cnt_0150")

Let's use yt to take a slice of density and temperature through the center of the dataset so we can see what we're looking at: 

In [ ]:
slc = yt.SlicePlot(ds, "z", ["density", "temperature"], width=(1.0,"Mpc"))
slc.show()

Ok, sloshing gas as advertised. Next, we'll create a sphere object to serve as a source for the photons. Place it at the center of the domain with `"c"`, and use a radius of 500 kpc:

In [ ]:
sp = ds.sphere("c", (500.,"kpc"))

Now, we need to set up a spectral model. We said we were going to look at the thermal emission from the hot plasma, so to do that we can set up a thermal spectrum. We'll do that here using `PyXspec` under the hood, by creating an `XSpecThermalModel`. The first argument is the model to use, and the next three arguments are the maximum and minimum energies, and the number of bins in the spectrum. We've chosen these numbers so that the spectrum has an energy resolution of about 1 eV. Specifying `"bapec"` as the thermal model automatically turns on thermal broadening. 

In [ ]:
spec_model = pyxsim.XSpecThermalModel("bapec", 0.05, 11.0, 10000)

Now that we've created a spectral model, we need to use it for our source model, which is that of a thermal source, buy using `ThermalSourceModel`. It takes a lot of different arguments, which you can investigate in the docs, but here we'll do something simple and say that it uses the above spectral model and the metallicity is a constant $Z = 0.3~Z_\odot$:

In [ ]:
source_model = pyxsim.ThermalSourceModel(spec_model, Zmet=0.3)

We're almost ready to go to generate the photons from this source, but first we should decide what our redshift, collecting area, and exposure time should be. Let's pick big numbers, because remember the point of this first step is to create a Monte-Carlo sample from which to draw smaller sub-samples for mock observations. Note these are all (value, unit) tuples:

In [ ]:
exp_time = (500., "ks") # exposure time
area = (2000.0, "cm**2") # collecting area
redshift = 0.05

So, that's everything--let's create the photons!

In [ ]:
photons = pyxsim.PhotonList.from_data_source(sp, redshift, area, exp_time, source_model)

Ok, that was easy. Now we have a `PhotonList` that we can use to create events. To be realistic, we're going to want to create an foreground Galactic absorption model, using the "TBabs" absorption mdoel in XSpec and assuming a foreground absorption column of $N_H = 4 \times 10^{20}~{\rm cm}^{-2}$:

In [ ]:
tbabs_model = pyxsim.XSpecAbsorbModel("TBabs", 0.04)

Once we have created this, we can project the photons. Here we'll just do a simple projection along the z-axis, reducing the exposure time, and centering the photons at RA, Dec = (45, 30) degrees:

In [ ]:
events_z = photons.project_photons("z", exp_time_new=(100.,"ks"), absorb_model=tbabs_model,
                                   sky_center=(45.,30.))

Finally, we'll convolve the events with the built-in *Chandra* ACIS-I instrument model, which re-bins the events, applies an energy-dependent cut on the observed photons using the effective area in the ARF, applies a Gaussian PSF, and convolves the energies with an RMF:

In [ ]:
events_acis = pyxsim.ACIS_I(events_z)

Now write an image of the events to disk, making an energy cut between 0.5 and 7.0 keV:

In [ ]:
events_acis.write_fits_image("sloshing_z_img.fits", emin=0.5, emax=7.0, clobber=True)

To take a quick look at the image, we'll use the [APLpy](https://aplpy.github.io/) Python FITS viewer:

In [ ]:
import aplpy
fig = aplpy.FITSFigure("sloshing_z_img.fits")
fig.show_colorscale(cmap='arbre', vmax=10, vmin=0.0, stretch='sqrt')
fig.recenter(45., 30., radius=0.1)